In [12]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from qucumber.nn_states import ComplexWaveFunction

from qucumber.callbacks import MetricEvaluator

import qucumber.utils.unitaries as unitaries
import qucumber.utils.cplx as cplx

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber

# set random seed on cpu but not gpu, since we won't use gpu for this tutorial
qucumber.set_random_seed(1234, cpu=True, gpu=False)

In [13]:
train_path = "qubits_train.txt"
train_bases_path = "qubits_train_bases.txt"
psi_path = "qubits_psi.txt"
bases_path = "qubits_bases.txt"

train_samples, true_psi, train_bases, bases = data.load_data(
    train_path, psi_path, train_bases_path, bases_path
)

In [14]:
unitary_dict = unitaries.create_dict()
# unitary_dict = unitaries.create_dict(<unitary_name>=torch.tensor([[real part],
#                                                                   [imaginary part]],
#                                                                  dtype=torch.double)

In [15]:
nv = train_samples.shape[-1]
nh = nv

nn_state = ComplexWaveFunction(
    num_visible=nv, num_hidden=nh, unitary_dict=unitary_dict, gpu=False
)

In [22]:
epochs = 1000
pbs = 100  # pos_batch_size
nbs = pbs  # neg_batch_size
lr = 0.1
k = 10

In [23]:
def alpha(nn_state, space, **kwargs):
    rbm_psi = nn_state.psi(space)
    normalization = nn_state.normalization(space).sqrt_()
    alpha_ = cplx.norm(
        torch.tensor([rbm_psi[0][0], rbm_psi[1][0]], device=nn_state.device)
        / normalization
    )

    return alpha_


def beta(nn_state, space, **kwargs):
    rbm_psi = nn_state.psi(space)
    normalization = nn_state.normalization(space).sqrt_()
    beta_ = cplx.norm(
        torch.tensor([rbm_psi[0][1], rbm_psi[1][1]], device=nn_state.device)
        / normalization
    )

    return beta_


def gamma(nn_state, space, **kwargs):
    rbm_psi = nn_state.psi(space)
    normalization = nn_state.normalization(space).sqrt_()
    gamma_ = cplx.norm(
        torch.tensor([rbm_psi[0][2], rbm_psi[1][2]], device=nn_state.device)
        / normalization
    )

    return gamma_


def delta(nn_state, space, **kwargs):
    rbm_psi = nn_state.psi(space)
    normalization = nn_state.normalization(space).sqrt_()
    delta_ = cplx.norm(
        torch.tensor([rbm_psi[0][3], rbm_psi[1][3]], device=nn_state.device)
        / normalization
    )

    return delta_

In [24]:
period = 25
space = nn_state.generate_hilbert_space()

callbacks = [
    MetricEvaluator(
        period,
        {
            "Fidelity": ts.fidelity,
            "KL": ts.KL,
            "normα": alpha,
            # "normβ": beta,
            # "normγ": gamma,
            # "normδ": delta,
        },
        target=true_psi,
        bases=bases,
        verbose=True,
        space=space,
    )
]

In [25]:
nn_state.fit(
    train_samples,
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    input_bases=train_bases,
    callbacks=callbacks,
    time=True,
)

Epoch: 25	Fidelity = 0.365628	KL = 0.928695	normα = 0.261269
Epoch: 50	Fidelity = 0.361265	KL = 0.950332	normα = 0.261781
Epoch: 75	Fidelity = 0.358392	KL = 0.943525	normα = 0.257624
Epoch: 100	Fidelity = 0.368020	KL = 0.937742	normα = 0.267112
Epoch: 125	Fidelity = 0.361861	KL = 0.949730	normα = 0.262770
Epoch: 150	Fidelity = 0.361275	KL = 0.944623	normα = 0.260534
Epoch: 175	Fidelity = 0.358464	KL = 0.951411	normα = 0.258830
Epoch: 200	Fidelity = 0.369098	KL = 0.937510	normα = 0.268680
Epoch: 225	Fidelity = 0.370611	KL = 0.923354	normα = 0.265718
Epoch: 250	Fidelity = 0.361770	KL = 0.945547	normα = 0.261106
Epoch: 275	Fidelity = 0.367904	KL = 0.937182	normα = 0.266554
Epoch: 300	Fidelity = 0.368150	KL = 0.921620	normα = 0.262502
Epoch: 325	Fidelity = 0.369764	KL = 0.935949	normα = 0.267899
Epoch: 350	Fidelity = 0.355212	KL = 0.958202	normα = 0.255516
Epoch: 375	Fidelity = 0.366644	KL = 0.943505	normα = 0.265819
Epoch: 400	Fidelity = 0.364750	KL = 0.933088	normα = 0.260925
Epoch: 425	

In [26]:
alpha(nn_state, space)

tensor(0.2561, dtype=torch.float64)

In [29]:
from qucumber.observables import SigmaZ
SigmaZ

qucumber.observables.pauli.SigmaZ

In [31]:
# Note that the key given to the *MetricEvaluator* must be
# what comes after callbacks[0].
fidelities = callbacks[0].Fidelity

# Alternatively, we may use the usual dictionary/list subscripting
# syntax. This is useful in cases where the name of the metric
# may contain special characters or spaces.
KLs = callbacks[0]["KL"]
coeffs = callbacks[0]["normα"]
epoch = np.arange(period, epochs + 1, period)

In [32]:
# Some parameters to make the plots look nice
params = {
    "text.usetex": True,
    "font.family": "serif",
    "legend.fontsize": 14,
    "figure.figsize": (10, 3),
    "axes.labelsize": 16,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "lines.linewidth": 2,
    "lines.markeredgewidth": 0.8,
    "lines.markersize": 5,
    "lines.marker": "o",
    "patch.edgecolor": "black",
}
plt.rcParams.update(params)
plt.style.use("seaborn-deep")

In [33]:
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(14, 3))
ax = axs[0]
ax.plot(epoch, fidelities, "o", color="C0", markeredgecolor="black")
ax.set_ylabel(r"Fidelity")
ax.set_xlabel(r"Epoch")

ax = axs[1]
ax.plot(epoch, KLs, "o", color="C1", markeredgecolor="black")
ax.set_ylabel(r"KL Divergence")
ax.set_xlabel(r"Epoch")

ax = axs[2]
ax.plot(epoch, coeffs, "o", color="C2", markeredgecolor="black")
ax.set_ylabel(r"$\vert\alpha\vert$")
ax.set_xlabel(r"Epoch")

plt.tight_layout()
plt.show()

KeyboardInterrupt: 

In [ ]:
nn_state.save("saved_params.pt")

In [ ]:
nn_state

In [ ]:
weights

In [34]:
nn_state.weights

Parameter containing:
tensor([[-0.0348,  0.1966],
        [-0.7583,  0.0821]], dtype=torch.float64)

In [25]:
nn_state.visible_bias

Parameter containing:
tensor([-0.0860, -0.1090], dtype=torch.float64)

In [26]:
nn_state.hidden_bias

Parameter containing:
tensor([-0.0077,  0.0112], dtype=torch.float64)

In [ ]:
nn_stat